<a href="https://colab.research.google.com/github/navanchauhan/Malaria-Detection/blob/master/notebooks/Turicreate_Obejct_Detection_Malaria_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection for Malaria using Turicreate

### 1) Mount Google Drive

The trained model is thus directly copied to your google drive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### 2) Install Dependencies

To train the model on the Colab GPU we need to install the custom version of mxnet


##### Make sure you have selected GPU in the runtype settings

In [0]:
!pip install turicreate
!pip uninstall -y mxnet
!pip install mxnet-cu100==1.4.0.post0

     |████████████████████████████████| 90.7MB 116kB/s 
     |████████████████████████████████| 23.8MB 168kB/s 
     |████████████████████████████████| 2.1MB 54.3MB/s 
     |████████████████████████████████| 327kB 66.5MB/s 
     |████████████████████████████████| 3.4MB 54.7MB/s 
     |████████████████████████████████| 17.3MB 214kB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320847 sha256=8f4647fe61c0750fdbedd81f2721b7efd09d7d5246666e796c607f2cd3365562
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built resampy
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requ

Uninstalling mxnet-1.1.0.post0:
  Successfully uninstalled mxnet-1.1.0.post0
     |████████████████████████████████| 487.9MB 32kB/s 
     |████████████████████████████████| 13.8MB 27.3MB/s 
ERROR: turicreate 5.8 requires mxnet<1.2.0,>=1.1.0, which is not installed.
ERROR: turicreate 5.8 has requirement numpy==1.16.4, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: spacy 2.1.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.59 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cvxpy 1.0.25 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.

Now restart the runtime.

### 3) Fetch the dataset

In [0]:
!wget http://air.ug/downloads/plasmodium-images.zip
!unzip plasmodium-images.zip
!ls images/

--2019-11-05 00:45:29--  http://air.ug/downloads/plasmodium-images.zip
Resolving air.ug (air.ug)... 34.242.164.142
Connecting to air.ug (air.ug)|34.242.164.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148185339 (141M) [application/zip]
Saving to: ‘plasmodium-images.zip’

plasmodium-images.z 100%[===================>] 141.32M  21.3MB/s    in 6.7s    

2019-11-05 00:45:35 (21.0 MB/s) - ‘plasmodium-images.zip’ saved [148185339/148185339]

Archive:  plasmodium-images.zip
  inflating: annotation/plasmodium-0000.xml  
  inflating: annotation/plasmodium-0001.xml  
  inflating: annotation/plasmodium-0002.xml  
  inflating: annotation/plasmodium-0003.xml  
  inflating: annotation/plasmodium-0004.xml  
  inflating: annotation/plasmodium-0005.xml  
  inflating: annotation/plasmodium-0006.xml  
  inflating: annotation/plasmodium-0007.xml  
  inflating: annotation/plasmodium-0008.xml  
  inflating: annotation/plasmodium-0009.xml  
  inflating: annotation/plasmodium-0

### 4) Fix issues with the dataset

In [0]:
import os
from xml.etree import ElementTree as ET

path = './annotation/'

replaceString = True
addFileName = True

def addname(file, xml_file):
	tree = ET.parse(xml_file)
	root = tree.getroot()

	f = ET.Element("filename")

	f.text = file

	root.insert(1, f)

	tree.write(xml_file)


def replaceIt(filename, old, new):
	f = open(filename,'r')
	filedata = f.read()
	f.close()
	newdata = filedata.replace(old,new)
	f = open(filename,'w')
	f.write(newdata)
	f.close()



for filename in os.listdir(path):
	if not filename.endswith('.xml'): continue
	fullname = os.path.join(path, filename)
	if addFileName:
		imagename = filename.replace('xml','jpg')
		addname(imagename, fullname)
	if replaceString:
		replaceIt(fullname, "person", "plasmodium")
	print("Fixed", fullname)



Fixed ./annotation/plasmodium-0492.xml
Fixed ./annotation/plasmodium-2168.xml
Fixed ./annotation/plasmodium-1707.xml
Fixed ./annotation/plasmodium-0149.xml
Fixed ./annotation/plasmodium-1330.xml
Fixed ./annotation/plasmodium-0485.xml
Fixed ./annotation/plasmodium-0756.xml
Fixed ./annotation/plasmodium-1670.xml
Fixed ./annotation/plasmodium-1875.xml
Fixed ./annotation/plasmodium-0108.xml
Fixed ./annotation/plasmodium-1053.xml
Fixed ./annotation/plasmodium-0659.xml
Fixed ./annotation/plasmodium-2069.xml
Fixed ./annotation/plasmodium-0153.xml
Fixed ./annotation/plasmodium-2155.xml
Fixed ./annotation/plasmodium-1284.xml
Fixed ./annotation/plasmodium-2352.xml
Fixed ./annotation/plasmodium-0261.xml
Fixed ./annotation/plasmodium-0195.xml
Fixed ./annotation/plasmodium-1939.xml
Fixed ./annotation/plasmodium-1430.xml
Fixed ./annotation/plasmodium-0935.xml
Fixed ./annotation/plasmodium-0041.xml
Fixed ./annotation/plasmodium-0605.xml
Fixed ./annotation/plasmodium-1007.xml
Fixed ./annotation/plasmo

### 5) Download pregenerated csv file

The annotations.csv file was created using RectLabel

In [0]:
!wget https://gist.githubusercontent.com/navanchauhan/dd91c089c743217868a00d68c9b7d3fe/raw/1b61cbc39d8a7406f2fbb2d687d34820aa063054/annotations.csv

--2019-11-05 00:45:49--  https://gist.githubusercontent.com/navanchauhan/dd91c089c743217868a00d68c9b7d3fe/raw/1b61cbc39d8a7406f2fbb2d687d34820aa063054/annotations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5091493 (4.9M) [text/plain]
Saving to: ‘annotations.csv’

annotations.csv     100%[===================>]   4.86M  --.-KB/s    in 0.04s   

2019-11-05 00:45:49 (137 MB/s) - ‘annotations.csv’ saved [5091493/5091493]



### 6) Change Absolute Path to Relative Path

In [0]:
def replaceIt(filename, old, new):
	f = open(filename,'r')
	filedata = f.read()
	f.close()
	newdata = filedata.replace(old,new)
	f = open(filename,'w')
	f.write(newdata)
	f.close()

replaceIt("./annotations.csv","/Users/navanchauhan/Downloads/malaria-data-object/plasmodium-images/images","./images")


### 7) Train Model

In [0]:
import turicreate as tc
import sys
import os

tc.config.set_num_gpus(-1)

csv_file = './annotations.csv'
csv = tc.SFrame(csv_file)

image_path = csv['path'][0]
image_folder = str(os.path.split(image_path)[0])

data = tc.image_analysis.load_images(image_folder, recursive=False)
data = data.join(csv)

train, val = data.random_split(0.8)

model = tc.object_detector.create(train, max_iterations=10000)


model.save('plasmodium.model')
model.export_coreml('plasmodium.mlmodel')

scores = model.evaluate(val)
print(scores)


Finished parsing file /content/annotations.csv

Parsing completed. Parsed 100 lines in 0.24289 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,list]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/annotations.csv

Parsing completed. Parsed 2703 lines in 0.24236 secs.

         To fix this, please install the currently recommended version:

             pip uninstall -y mxnet && pip install mxnet==1.1.0

         If you want to use a CUDA GPU, then change 'mxnet' to 'mxnet-cu90' (adjust 'cu90' depending on your CUDA version):

Using 'image' as feature column
Using 'annotations' as annotations column
Download completed: /var/tmp/model_cache/darknet.params
Setting 'batch_size' to 32
Using GPU to create model (Tesla P100-PCIE-16GB)
+--------------+--------------+--------------+
| Iteration    | Loss         | Elapsed Time |
+--------------+--------------+--------------+
| 1            | 11.321       | 14.5         |
| 6            | 11.964       | 26.9         |
| 11           | 11.867       | 39.4         |
| 15           | 11.222       | 49.7         |
| 19           | 11.003       | 60.2         |
| 23           | 10.262       | 70.4         |
| 28           | 9.847        | 82.6         |
| 32           | 9.412        | 92.8         |
| 37          

### 8) Exporting Model for Conversion

ONNX currently does not support non maximum suppression 

In [0]:
model.export_coreml('plasmodium-for-onnx.mlmodel', include_non_maximum_suppression=False)

### 9) Save to Google Drive


In [0]:
!cp "./plasmodium-for-onnx.mlmodel" "/content/gdrive/My Drive/."

### 10) Installing ONNX and converting model

In [0]:
!pip install onnx coremltools onnxmltools

In [0]:
import onnxmltools
import coremltools

In [0]:
input_coreml_model = 'plasmodium-for-onnx.mlmodel'
output_onnx_model = 'model.onnx'

In [0]:
coreml_model = coremltools.utils.load_spec(input_coreml_model)
onnx_model = onnxmltools.convert_coreml(coreml_model)
onnxmltools.utils.save_model(onnx_model, output_onnx_model)

!cp ./model.onnx "/content/gdrive/My Drive/."